## Functional Programming

* refers to a programming paradigm where we treat a function as an object
* we define our logic using those function objects.
* it mostly based on two important function concepts
    1. we can pass a function to another as an argument.
    2. a function can return another function as an argument
    
#### Calculator Example


In [8]:
class Calculator:
    def present(self,value1, opr, value2):
        if opr == 'plus':
            result= value1 + value2
        elif opr == 'minus':
            result= value1 - value2
        elif opr == 'multiply':
            result= value1 * value2
        elif opr == 'divide':
            result= value1 / value2
        else:
            raise ValueError(f"Invalid operator:{opr}")

        print(f'{value1} {opr} {value2} = {result}')    
    

In [9]:
calc=Calculator()

calc.present(20, 'plus', 30)
calc.present(20, 'divide', 4)


20 plus 30 = 50
20 divide 4 = 5.0


In [10]:
calc.present(20, 'mod', 7)

ValueError: Invalid operator:mod

#### Problem

* we have a working calculator
* but the calculator is not extensible
* if we need more operations tomorrow (mod,power, permutation) we will have to modify the calculator again.
* similarly current calculator always displays the result on console
    * if tomorrow we want to display the result at different place (web page, GUI) this calcualtor will be useless, although it will still can peform the calculation

* this code has different responsibilities. mixed


#### Solution ---> Function Object

* treat each operation as function object
* pass the function object you want to use.
* also treat the print() function as one of the possible choices for presenting output.
* we can replace print with any other function of our choice.

#### Phase #1 create each operation as a separate function

In [11]:
def plus(a,b): return a+b
def minus(a,b): return a-b
def multiply(a,b): return a*b
def divide(a,b): return a/b

### Phase #2 calculator can take these (and more) functions as parameter

#### Approach #1
* user can pass the function that it wants to invoke

In [13]:
class Calculator:
    def present(self,value1, opr, value2):
        result=opr(value1,value2)
        print(f'{value1} {opr.__name__} {value2} = {result}') 

In [14]:
calc=Calculator()
calc.present(20, plus, 30)
calc.present(20, divide, 11)
# now you can't really pass a function, you don't have

20 plus 30 = 50
20 divide 11 = 1.8181818181818181


#### Problem: client may not be able to pass the function directly
* can we pass a string?

In [20]:
class Calculator:
    def __init__(self, presenter=None):
        self.presenter=presenter
        if self.presenter is None:
            self.presenter= lambda v1,opr,v2,result: print(f'{v1} {opr} {v2} => {result}')

        self._operators={}
        self._add_basic_operators()

    def _add_basic_operators(self):
        self.add_operator(plus)
        self.add_operator(minus)
        self.add_operator(multiply)
        self.add_operator(divide)


    def add_operator(self, operator, name=None):
        if name is None:
            name = operator.__name__
        self._operators[name.lower()]=operator

    def present(self,value1, opr, value2):
        opr=opr.lower()
        if opr in self._operators:
            fn=self._operators[opr]
            result=fn(value1,value2)
            print(f'{value1} {opr} {value2} = {result}')
        else:
            raise ValueError(f'No such operator: "{opr}"') 

In [21]:
calc=Calculator()

calc.present(20, 'plus', 30)

calc.present(20, 'Divide', 11)

20 plus 30 = 50
20 divide 11 = 1.8181818181818181


In [22]:
calc.present(20, 'mod',7)

ValueError: No such operator: "mod"

### But we can add mod logic dynamically

In [23]:
def mod(v1,v2): return v1%v2

calc.add_operator(mod)

In [24]:
calc.present(20, 'mod', 7)

20 mod 7 = 6


### we can also add functionality use lambda function

* but all lambdas are called lambda
* so we will a friendly name in add_operator function as optional second argument

In [26]:
calc.add_operator( lambda v1,v2: v1**v2, 'power')

In [27]:
calc.present(20, 'power', 3)

20 power 3 = 8000


#### Assignment 2.3

* write a search function that takes a sequence of values and returns
    * all even numbers
    * all prime numbers
    * all numbers divisible by 7
    * all permanent employees of an organization

* search should be your function
    * No built-in function use allowed.

### Solution #1

In [1]:
def search( values):
    result=[]
    for value in values:
        if value %2 ==0:
            result.append(value)
    return result

In [2]:
values=[2,9,11,8,4,13,3,72,8,4,25,33,27]



In [3]:
search(values)

[2, 8, 4, 72, 8, 4]

#### Best Practices

*  search shouldn't return a list.
    * it should yield.
    * never return a large list if it can be avoided.

In [4]:
def search( values):
    
    for value in values:
        if value %2 ==0:
            yield(value)
    

In [5]:
for evens in search(values):
    print(evens,  end=' ')

2 8 4 72 8 4 

### Problem --> it only searches evens.

* what about other requiremens like
    * search prime
    * search employees


### Approach #1

In [2]:
def search_evens( values):
    
    for value in values:
        if value %2 ==0:
            yield(value)

def is_prime(value):
    if value<2: return False
    for i in range(2,value):
        if value %i ==0:
            return False
    return True

def search_prime(values):
    
    for value in values:
        if is_prime(value):
            yield(value)

In [7]:
for prime in search_prime(values):
    print(prime, end=' ')

2 11 13 3 

In [11]:
import org

bosch= org.get_organization('Bosch',50)


### Search logic for permanent employees

In [7]:
def search_permanent_employees(org):
    for employee in org:
        if employee._isPermanent:
            yield employee

In [10]:
count=0
for e in search_permanent_employees(bosch):
    count+=1
    
print(count)

32


### Problem

* we are writing multiple search functions
* They have largely common code
* they differ just at point

### Approach #2

In [15]:
def search(values,criteria):
    results=[]
    for value in values:
        match=False
        if criteria=='evens':
            match= value%2==0
        elif criteria=='prime':
            match= is_prime(value)
        elif criteria=='permanent_employees':
            match= value._isPermanent
        
        if match:
            results.append(value)

    return results

In [16]:
def print_results(results):
    for result in results:
        print(result, end=' ')

In [18]:
values=[2,9,11,4,15,18,7,61,8]
print_results(search(values,'evens'))

2 4 18 8 

In [19]:
print_results(search(values,'prime'))

2 11 7 61 

In [21]:
result= search(bosch,'permanent_employees')
count=0
for emp in result:
    count+=1
    print(emp._name, emp._isPermanent )

print('total ',count)

Employee3 True
Employee5 True
Employee6 True
Employee9 True
Employee10 True
Employee12 True
Employee13 True
Employee15 True
Employee16 True
Employee17 True
Employee18 True
Employee20 True
Employee22 True
Employee24 True
Employee25 True
Employee28 True
Employee29 True
Employee30 True
Employee32 True
Employee33 True
Employee34 True
Employee35 True
Employee36 True
Employee37 True
Employee38 True
Employee39 True
Employee40 True
Employee41 True
Employee42 True
Employee43 True
Employee44 True
Employee46 True
Employee48 True
total  33


### problem

* just like calculator we have multiple logic merged in one function
* if tomorrow we need more criteria we will have to write logic for each of them in the search

* remember each crieria in iteself is a function
    * can be represented with a function object

#### Passing a callback function

In [23]:
def search(values, criteria):
    results=[]
    for value in values:
        if criteria(value):
            results.append(value)

    return results

In [24]:
primes = search(values, is_prime)

for prime in primes:
    print(prime, end=' ')

2 11 7 61 

### to perform even search we can just define is_even function

In [25]:
def is_even(value): return value%2==0

In [26]:
evens = search(values, is_even)
print_results(evens)

2 4 18 8 

### Or we can use lambda function to pass search criteria

In [27]:
permanent = search(bosch, lambda emp: emp._isPermanent)
for emp in permanent:
    print(emp._name, end='\t')

Employee3	Employee5	Employee6	Employee9	Employee10	Employee12	Employee13	Employee15	Employee16	Employee17	Employee18	Employee20	Employee22	Employee24	Employee25	Employee28	Employee29	Employee30	Employee32	Employee33	Employee34	Employee35	Employee36	Employee37	Employee38	Employee39	Employee40	Employee41	Employee42	Employee43	Employee44	Employee46	Employee48	

### Now we can do different types of searches


#### Find all employess whose salary is less that 35000

In [31]:
result = search(bosch, lambda emp: emp._salary<35000)

for emp in result:
    print(emp._name, emp._salary)

Employee0 28000
Employee2 21000
Employee3 31000
Employee6 32000
Employee7 20000
Employee9 23000
Employee14 23000
Employee16 21000
Employee18 20000
Employee20 23000
Employee21 25000
Employee22 33000
Employee24 30000
Employee27 23000
Employee32 34000
Employee36 25000
Employee41 29000
Employee42 31000
Employee43 23000
Employee44 23000
Employee47 34000
Employee49 30000
